# A simple example of generating playlist by multilable learning (toppush)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os, sys, time
import pickle as pkl
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, f1_score, make_scorer, label_ranking_loss
from scipy.sparse import lil_matrix, issparse

import matplotlib.pyplot as plt
import seaborn as sns
#import inspect

In [ ]:
sys.path.append('src')
from PClassificationMLC import PClassificationMLC
from evaluate import evaluatePrecision, evalPred, avgPrecisionK, f1_score_nowarn
from BinaryRelevance import BinaryRelevance

In [ ]:
data_dir = 'data'
faotm = os.path.join(data_dir, 'aotm-2011/aotm-2011-subset.pkl')
#fmap  = os.path.join(data_dir, 'aotm-2011/songID2TrackID.pkl')
ffeature = os.path.join(data_dir, 'msd/songID2Features.pkl')

In [ ]:
fx      = os.path.join(data_dir, 'aotm-2011/X_audio.pkl')
fy      = os.path.join(data_dir, 'aotm-2011/Y_audio.pkl')
fxtrain = os.path.join(data_dir, 'aotm-2011/X_train_audio.pkl')
fytrain = os.path.join(data_dir, 'aotm-2011/Y_train_audio.pkl')
fxtest  = os.path.join(data_dir, 'aotm-2011/X_test_audio.pkl')
fytest  = os.path.join(data_dir, 'aotm-2011/Y_test_audio.pkl')

## Data loading

Load playlists.

In [ ]:
playlists = pkl.load(open(faotm, 'rb'))

In [ ]:
print('#Playlists: %d' % len(playlists))

In [ ]:
playlists[0]

In [ ]:
#print('#Songs: %d' % len({songID for p in playlists for songID in p['filtered_lists'][0]}))

In [ ]:
#lengths = [len(p['filtered_lists'][0]) for p in playlists]
lengths = [len(sl) for sl in playlists]
plt.hist(lengths, bins=20)
print('Average playlist length: %.1f' % np.mean(lengths))

Load `song_id` --> `track_id` mapping: a song may correspond to multiple tracks.

In [ ]:
#song2TrackID = pkl.load(open(fmap, 'rb'))

In [ ]:
#{ k : song2TrackID[k] for k in list(song2TrackID.keys())[:10] }

Load `song_id` --> `feature array` mapping: map a song to the audio features of one of its corresponding tracks in MSD.

In [ ]:
song2Features = pkl.load(open(ffeature, 'rb'))

The set of songs, which is the set of labels in this formulation.

In [ ]:
#song_set = sorted(song2Features.keys())  # use MSD songs as label space
song_set = sorted({sid for pl in playlists for sid in pl})   # use the intersection of MSD and AotM as label space

In [ ]:
len(song_set)

In [ ]:
label_indices = {songID: ix for ix, songID in enumerate(song_set)}

In [ ]:
list(label_indices.items())[:10]

In [ ]:
def gen_training_set(playlists=playlists, label_indices=label_indices, features=song2Features):
    """
        Create the labelled dataset for a given song index
        
        Input:
            - playlists: which playlists to create features for
            - label_indices: a dictionary that maps a songID to the index of the corresponding label
            - features: a dictionary that maps a songID to its feature vector
            
        Output:
            - (Feature, Label) pair (X, Y), with # num playlists rows
              X comprises the features for each seed song
              Y comprises the indicators of whether the given song is present in the respective playlist
    """

    N = len(playlists)
    K = len(label_indices)

    X = [ ]
    Y = lil_matrix((N, K), dtype=np.int8)
    
    cnt = 0
    for i in range(len(playlists)):
        cnt += 1
        if cnt % 100 == 0:
            sys.stdout.write('\r%d / %d' % (cnt, len(playlists)))
            sys.stdout.flush()
            
        playlist = playlists[i]
        seed     = playlist[0]

        X.append(features[seed])
        #indices = [label_indices[s] for s in playlist]
        indices = [label_indices[s] for s in playlist if s in label_indices]
        Y[i, indices] = 1

    return np.array(X), Y.tocsr()

In [ ]:
test_dict = {1: 0, 2: 1, 3: 2}
[test_dict[s] for s in [1, 2, 5] if s in test_dict]

## Training & Test

Train a logistic regression model for each label.

In [ ]:
if np.all([os.path.exists(fname) for fname in [fxtrain, fytrain, fxtest, fytest]]):
    X_train = pkl.load(open(fxtrain, 'rb'))
    Y_train = pkl.load(open(fytrain, 'rb'))
    X_test  = pkl.load(open(fxtest,  'rb'))
    Y_test  = pkl.load(open(fytest,  'rb'))
else:
    X, Y = gen_training_set(playlists=playlists, label_indices=label_indices, features=song2Features)
    # by fixing random seed, the same playlists will be in the test set each time
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=31)
    pkl.dump(X,       open(fx,      'wb'))
    pkl.dump(Y,       open(fy,      'wb'))
    pkl.dump(X_train, open(fxtrain, 'wb'))
    pkl.dump(Y_train, open(fytrain, 'wb'))
    pkl.dump(X_test,  open(fxtest,  'wb'))
    pkl.dump(Y_test,  open(fytest,  'wb'))

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
Y_train.shape

In [ ]:
Y_test.shape

Feature normalisation.

In [ ]:
X_train_mean = np.mean(X_train, axis=0).reshape((1, -1))
X_train_std = np.std(X_train, axis=0).reshape((1, -1)) + 10 ** (-6)
X_train -= X_train_mean
X_train /= X_train_std
X_test  -= X_train_mean
X_test  /= X_train_std

In [ ]:
def avgF1(Y_true, Y_pred):
    F1 = f1_score_nowarn(Y_true, Y_pred >= 0, average='samples')
    print('F1: %g, #examples: %g' % (F1, Y_true.shape[0]))
    return F1

Train.

In [ ]:
clf = PClassificationMLC(C=100, p=2, weighting=True)
clf.fit_SGD(X_train, Y_train, batch_size=200, n_epochs=20, learning_rate=0.05)

In [ ]:
def dumpclf(clf, batch_size, n_epochs, learning_rate):
    C = clf.C
    p = clf.p
    weighting = clf.weighting
    fname = 'model-%d-%g-%s-%d-%d-%g.pkl' % (C, p, weighting, batch_size, n_epochs, learning_rate)
    w = np.concatenate((clf.b, clf.W.ravel()), axis=-1)
    pkl.dump(w, open(fname, 'wb'))

In [ ]:
dumpclf(clf, batch_size=200, n_epochs=10, learning_rate=0.05)

In [ ]:
fname = 'model-%d-%g-%s-%d-%d-%g.pkl' % (100, 2, True, 200, 10, 0.05)
W0 = pkl.load(open(fname, 'rb'))

In [ ]:
clf1 = PClassificationMLC(C=100, p=2, weighting=True)
clf1.fit_SGD(X_train, Y_train, w=w0, batch_size=200, n_epochs=10, learning_rate=0.05)

In [ ]:
#print(avgF1(Y_train, clf.decision_function(X_train)))
#print(avgF1(Y_test, clf.decision_function(X_test)))

In [ ]:
#inspect.signature(evaluate).parameters.keys()

In [ ]:
#'threshold' in inspect.signature(evaluate).parameters.keys()

In [ ]:
def evaluate(clf, eval_func, X_test, Y_test, threshold=None, batch_size=100):
    assert X_test.shape[0] == Y_test.shape[0]
    
    N = X_test.shape[0]
    metrics_all = []
    n_batches = int((N-1) / batch_size) + 1
    indices = np.arange(N)
    
    for nb in range(n_batches):
        sys.stdout.write('\r %d / %d' % (nb+1, n_batches))
        sys.stdout.flush()
        
        ix_start = nb * batch_size
        ix_end = min((nb+1) * batch_size, N)
        ix = indices[ix_start:ix_end]
        
        X = X_test[ix]
        Y_true = Y_test[ix]
        if issparse(Y_true):
            Y_true = Y_true.toarray()
        Y_pred = clf.decision_function(X)
        if issparse(Y_pred):
            Y_pred = Y_pred.toarray()
        if threshold is not None:
            Y_pred = Y_pred >= threshold
            
        metrics = eval_func(Y_true, Y_pred)
        metrics_all = np.concatenate((metrics_all, metrics), axis=-1)
        
    return metrics_all

In [ ]:
def calcF1(Y_true, Y_pred):
    """
    Compute F1 scores for multilabel prediction, one score for each example.
    precision = true_positive / n_true
    recall = true_positive / n_positive
    f1 = (2 * precision * recall) / (precision + recall) = 2 * true_positive / (n_true + n_positive)
    """
    assert Y_true.shape == Y_pred.shape
    N, K = Y_true.shape
    OneK = np.ones(K)
    
    n_true = np.dot(Y_true, OneK)
    n_positive = np.dot(Y_pred, OneK)    
    true_positive = np.dot(np.multiply(Y_true, Y_pred), OneK)
    
    numerator = 2 * true_positive
    denominator = n_true + n_positive
    nonzero_ix = np.nonzero(denominator)[0]
    
    f1 = np.zeros(N)
    f1[nonzero_ix] = np.divide(numerator[nonzero_ix], denominator[nonzero_ix])
    
    return f1

In [ ]:
metrics = evaluate(clf=clf, eval_func=calcF1, X_test=X_test, Y_test=Y_test, threshold=0.1, batch_size=500)

In [ ]:
np.mean(metrics)

In [ ]:
THs = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85]

In [ ]:
for th in THs:
    metrics = evaluate(clf=clf, eval_func=calcF1, X_test=X_test, Y_test=Y_test, threshold=th, batch_size=500)
    print(' Threshold: %.2g, F1: %g' % (th, np.mean(metrics)))

In [ ]:
def precisionatK(Y_true, Y_pred):
    assert 

In [ ]:
#ranges = range(-6, 7)
#ranges = range(-6, 5)
#parameters = [{'C': sorted([10**(e) for e in ranges] + [3 * 10**(e) for e in ranges]),
               #'r': [0.5, 1, 2, 4]}]
#scorer = {'Prec': make_scorer(avgPrecisionK)}

In [ ]:
clf1 = GridSearchCV(TopPushMLC(), parameters, scoring=scorer, cv=5, n_jobs=1, refit='Prec')
clf1.fit(X1_train, Y1_train)

In [ ]:
br1 = GridSearchCV(BinaryRelevance(), param_grid=[{'C': parameters[0]['C']}], scoring=scorer, \
                   cv=5, n_jobs=4, refit='Prec')
br1.fit(X1_train, Y1_train)

In [ ]:
print('TP1:')
evaluatePrecision(Y1_test, clf1.decision_function(X1_test))

In [ ]:
print('BR1:')
evaluatePrecision(Y1_test, br1.decision_function(X1_test))

### Type2 songs: popularities are somewhat uniform

In [ ]:
type2_songs = np.asarray(song_set)[indices[200:400]]

In [ ]:
type2_song_features = {sid: song2Features[sid] for sid in type2_songs}

In [ ]:
type2_song_label_indices = {sid: ix for ix, sid in enumerate(type2_songs)}

In [ ]:
playlist_subset2 = [pl for pl in playlists if pl[0] in type2_song_label_indices]

In [ ]:
X2, Y2 = gen_training_set(playlists=playlist_subset2, label_indices=type2_song_label_indices, \
                          features=type2_song_features)

Fitering out playlists with only one song.

In [ ]:
Y2 = Y2.toarray()

In [ ]:
ind2 = Y2.sum(axis=1) > 1

In [ ]:
X2, Y2 = X2[ind2], Y2[ind2]

In [ ]:
Y2.shape

Length histogram.

In [ ]:
pd.Series(np.sum(Y2, axis=1)).hist()

Popularity histogram.

In [ ]:
pd.Series(np.sum(Y2, axis=0)).hist()

In [ ]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size=0.33, random_state=7)

In [ ]:
X2_train_mean = np.mean(X2_train, axis=0).reshape((1, -1))
X2_train_std = np.std(X2_train, axis=0).reshape((1, -1)) + 10 ** (-6)
X2_train -= X2_train_mean
X2_train /= X2_train_std
X2_test  -= X2_train_mean
X2_test  /= X2_train_std

Train.

In [ ]:
clf2 = GridSearchCV(TopPushMLC(), parameters, scoring=scorer, cv=5, n_jobs=1, refit='Prec')
clf2.fit(X2_train, Y2_train)

In [ ]:
br2 = GridSearchCV(BinaryRelevance(), param_grid=[{'C': parameters[0]['C']}], scoring=scorer, \
                   cv=5, n_jobs=4, refit='Prec')
br2.fit(X2_train, Y2_train)

In [ ]:
print('TP2:')
evaluatePrecision(Y2_test, clf2.decision_function(X2_test))

In [ ]:
print('BR2:')
evaluatePrecision(Y2_test, br2.decision_function(X2_test))

In [ ]:
def print_results(predictor, X_train, Y_train, X_test, Y_test, trainPerf=False):
    """
        Compute and save performance results
    """
    batch_size = 500
    njobs = 16
    
    p3_test = []
    p5_test = []
    pk_test = []
    p10_test = []
    #rankloss_test = []
    
    N_test = X_test.shape[0]
    N_batch_test = int((N_test-1) / batch_size) + 1
    for i in range(N_batch_test):
        sys.stdout.write('\r%d / %d' % (i+1, N_batch_test)); sys.stdout.flush()
        ix0 = i * batch_size
        ix1 = min((i+1) * batch_size, N_test)
        preds = predictor.decision_function(X_test[ix0:ix1])
        evaldict = evaluatePrecision(Y_test[ix0:ix1].toarray(), preds, verbose=-1, n_jobs=njobs)
        size = ix1 - ix0
        p3_test.append(evaldict['Precision@3'][0] * size)
        p5_test.append(evaldict['Precision@5'][0] * size)
        pk_test.append(evaldict['Precision@K'][0] * size)
        p10_test.append(evaldict['Precision@10'][0] * size)
        #rankloss_test.append(evalPred1(Y_test[i].toarray()[0], pred, metricType='Ranking'))
    print()
    print('Test set:')
    print('Precision@3:', (np.sum(p3_test) / N_test))
    print('Precision@5:', (np.sum(p5_test) / N_test))
    print('Precision@k:', (np.sum(pk_test) / N_test))
    print('Precision@10:', (np.sum(p10_test) / N_test))
    print()
    
    if trainPerf is True:
        p3_train = []
        p5_train = []
        pk_train = []
        p10_train = []
        #rankloss_train = []

        N_train = X_train.shape[0]
        N_batch_train = int((N_train-1) / batch_size) + 1
        for i in range(N_batch_train):
            sys.stdout.write('\r%d / %d' % (i+1, N_batch_train)); sys.stdout.flush()
            ix0 = i * batch_size
            ix1 = min((i+1) * batch_size, N_train)
            preds = predictor.decision_function(X_train[ix0:ix1])
            evaldict = evaluatePrecision(Y_train[ix0:ix1].toarray(), preds, verbose=-1, n_jobs=njobs)
            size = ix1 - ix0
            p3_train.append(evaldict['Precision@3'][0] * size)
            p5_train.append(evaldict['Precision@5'][0] * size)
            pk_train.append(evaldict['Precision@K'][0] * size)
            p10_train.append(evaldict['Precision@10'][0] * size)
            #rankloss_train.append(evalPred1(Y_train[i].toarray()[0], pred, metricType='Ranking'))
        print()
        print('Training set:')
        print('Precision@3:', (np.sum(p3_train) / N_train))
        print('Precision@5:', (np.sum(p5_train) / N_train))
        print('Precision@k:', (np.sum(pk_train) / N_train))
        print('Precision@10:', (np.sum(p10_train) / N_train))
    
    #print()
    #print('Training set:')
    #print('RankingLoss: %.1f, %.1f' % (np.mean(rankloss_train), np.std(rankloss_train) / N_train))
    #print()
    #print('Test set:')
    #print('RankingLoss: %.1f, %.1f' % (np.mean(rankloss_test), np.std(rankloss_test) / N_test))

In [ ]:
def print_dataset_info(X_train, Y_train, X_test, Y_test):
    N_train, D = X_train.shape
    K = Y_train.shape[1]
    N_test = X_test.shape[0]
    print('%-45s %s' % ('Number of training examples:', '{:,}'.format(N_train)))
    print('%-45s %s' % ('Number of test examples:', '{:,}'.format(N_test)))
    print('%-45s %s' % ('Number of features:', '{:,}'.format(D)))
    print('%-45s %s' % ('Number of labels:', '{:,}'.format(K)))
    avgK_train = np.mean(np.sum(Y_train, axis=1))
    avgK_test  = np.mean(np.sum(Y_test, axis=1))
    print('%-45s %.3f (%.3f%%)' % ('Average number of positive labels (train):', avgK_train, 100*avgK_train / K))
    print('%-45s %.3f (%.3f%%)' % ('Average number of positive labels (test):', avgK_test, 100*avgK_test / K))
    #print('%-45s %.4f%%' % ('Average label occurrence (train):', np.mean(np.sum(Y_train, axis=0)) / N_train))
    #print('%-45s %.4f%%' % ('Average label occurrence (test):', np.mean(np.sum(Y_test, axis=0)) / N_test))
    print('%-45s %.3f%%' % ('Sparsity (percent) (train):', 100 * np.sum(Y_train) / np.prod(Y_train.shape)))
    print('%-45s %.3f%%' % ('Sparsity (percent) (test):', 100 * np.sum(Y_test) / np.prod(Y_test.shape)))

In [ ]:
print_dataset_info(X_train, Y_train, X_test, Y_test)

In [ ]:
clf_ = TopPushMLC(C=10000, r=2)
clf_.fit_SGD(X_train, Y_train, batch_size=500, n_epochs=10, learning_rate=0.05)

In [ ]:
print_results(clf_, X_train, Y_train, X_test, Y_test)